In [2]:
%matplotlib qt
from scipy.ndimage.measurements import label, find_objects
import numpy as np
import matplotlib.pyplot as plt

In [3]:
cd "C:\Users\jdavies\Google Drive\MAGPIE\data\2015\s1029_15 Reconnection C 32 Wire TS in layer 280 ns"

C:\Users\jdavies\Google Drive\MAGPIE\data\2015\s1029_15 Reconnection C 32 Wire TS in layer 280 ns


In [4]:
t=plt.imread('s1029_15 532 plasma.png')
t=t.sum(axis=2)
th=-np.clip(t,a_min=-10, a_max=2)+2

In [5]:
structure=np.ones((3,3))
labeled_array, num_fringes = label(th, structure=structure)
la=np.ma.masked_equal(labeled_array, 0)

In [27]:
fig, ax= plt.subplots()
ii=ax.imshow(-np.ma.masked_equal(la, 0)/300.0, interpolation='none')

In [22]:
%matplotlib qt
from matplotlib import pyplot as plt

class FringeNumberer:
    def __init__(self, im, num_fringes):
        self.num_fringes=num_fringes
        self.fringe_dict={key: np.nan for (key, value) in zip(range(1,num_fringes+1),range(num_fringes))}
        self.line,=im.axes.plot([0],[0],lw=2)
        self.im = im
        self.fringe_indexes= im.get_array().copy()
        self.update_fringe_map()
        self.xs = []
        self.ys = []
        self.in_a_line=False
        self.cid = im.figure.canvas.mpl_connect('button_press_event', self)

    def __call__(self, event):
        ax=self.im.axes
        print('click', event)
        if event.inaxes!=ax: 
            print('return')
            return
        if event.dblclick is True and self.in_a_line is False:
            print('Begin Line')
            ax.set_title('Dbl click to start, click to add, dbl click to end')
            self.in_a_line=True
            self.xs=[event.xdata]
            self.ys=[event.ydata]
            return
        if event.dblclick is True and self.in_a_line is True:
            print('End Line')
            self.in_a_line=False
            self.xs.append(event.xdata)
            self.ys.append(event.ydata)
            self.line.set_data(self.xs, self.ys)
            ax.set_xlim([0,la.shape[0]])
            ax.set_xlim([0,la.shape[1]])
            self.get_labels()
            self.check_labels()
            self.label_fringes()
            self.update_fringe_map()
            self.line.figure.canvas.draw()
            return
        if event.dblclick is False and self.in_a_line is True:
            print('Add segment')
            self.xs.append(event.xdata)
            self.ys.append(event.ydata)
            self.line.set_data(self.xs, self.ys)
            ax.set_xlim([0,la.shape[0]])
            ax.set_xlim([0,la.shape[1]])
            self.line.figure.canvas.draw()
            return
        
    def get_labels(self):
        d=self.fringe_indexes
        self.fringe_indexes_on_line=[]
        for i in range(len(self.xs)-1):
            x0, y0 = self.xs[i], self.ys[i] # These are in _pixel_ coordinates!!
            x1, y1 = self.xs[i+1], self.ys[i+1]
            length = 3*int(np.hypot(x1-x0, y1-y0))
            x, y = np.linspace(x0, x1, length), np.linspace(y0, y1, length)
            # Extract the values along the line
            zi = d[y.astype(np.int), x.astype(np.int)]
            zi=[z for z in zi if z!=0]
            self.fringe_indexes_on_line.extend(zi)
            
    def check_labels(self):        
        fn=self.fringe_indexes_on_line
        #remove adjacent duplicates
        self.fringe_indexes_on_line=[n for i, n in enumerate(fn) if i==0 or n != fn[i-1]]
        #check there are no other duplicates
        if len(set(self.fringe_indexes_on_line)) is not len(self.fringe_indexes_on_line):
            self.im.axes.set_title('Line recrosses fringes, try again')
            self.im.axes.title.set_color('r')
            self.line.figure.canvas.draw()
            self.fringe_indexes_on_line=[]
            return
        else:
            return
        
    def label_fringes(self):
        mode=+1
        start_index=self.fringe_dict[self.fringe_indexes_on_line[0]]
        if start_index is np.nan:
            start_index=1
        for ind, f in enumerate(self.fringe_indexes_on_line):
            self.fringe_dict[f]=start_index+ind*mode
            
    def update_fringe_map(self):
        arr=self.fringe_indexes.copy()
        for f in range(1,self.num_fringes+1):
            if self.fringe_dict[f] is np.nan:
                arr[arr==f]=1
            else:
                arr[arr==f]=self.fringe_dict[f]
        self.im.set_data(arr)
        self.im.set_clim([arr.min(), arr.max()])
        self.im.figure.canvas.draw()   
        
        
            
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title('Dbl click to start, click to add, dbl click to end')

cmap = plt.cm.jet
cmap.set_bad('w',0)
im = ax.imshow(la, cmap=cmap, interpolation='nearest')

ax.set_xlim([0,la.shape[0]])
ax.set_xlim([0,la.shape[1]])

FN = FringeNumberer(im, num_fringes)

plt.show()

click MPL MouseEvent: xy=(132,232) xydata=(325.790794979,1573.22175732) button=1 dblclick=False inaxes=Axes(0.157585,0.1;0.70983x0.8)
click MPL MouseEvent: xy=(132,232) xydata=(325.790794979,1573.22175732) button=1 dblclick=True inaxes=Axes(0.157585,0.1;0.70983x0.8)
Begin Line
click MPL MouseEvent: xy=(520,241) xydata=(4384.36820084,1479.07949791) button=1 dblclick=False inaxes=Axes(0.157585,0.1;0.70983x0.8)
Add segment
click MPL MouseEvent: xy=(520,241) xydata=(4384.36820084,1479.07949791) button=1 dblclick=True inaxes=Axes(0.157585,0.1;0.70983x0.8)
End Line
click MPL MouseEvent: xy=(569,601) xydata=(461.793357934,1227.8597786) button=1 dblclick=False inaxes=Axes(0.24421,0.1;0.53658x0.8)
click MPL MouseEvent: xy=(789,689) xydata=(1293.97665456,1431.69525949) button=1 dblclick=False inaxes=Axes(0.125,0.240339;0.775x0.519322)
click MPL MouseEvent: xy=(356,546) xydata=(1349.1613422,1515.4359754) button=1 dblclick=False inaxes=Axes(0.125,0.333925;0.775x0.332151)
click MPL MouseEvent: xy=(

In [15]:
im.set_clim()

2

In [24]:
FN.fringe_dict

{1: 68,
 2: 69,
 3: 70,
 4: 67,
 5: 13,
 6: nan,
 7: 14,
 8: 15,
 9: 16,
 10: 21,
 11: 17,
 12: 18,
 13: 20,
 14: 19,
 15: 71,
 16: 66,
 17: 12,
 18: 72,
 19: 65,
 20: nan,
 21: 11,
 22: 22,
 23: nan,
 24: nan,
 25: 64,
 26: 10,
 27: 63,
 28: 23,
 29: 62,
 30: 24,
 31: 9,
 32: 25,
 33: nan,
 34: 61,
 35: 60,
 36: 8,
 37: nan,
 38: nan,
 39: nan,
 40: nan,
 41: 26,
 42: nan,
 43: nan,
 44: nan,
 45: 7,
 46: 59,
 47: 27,
 48: nan,
 49: nan,
 50: nan,
 51: 58,
 52: 28,
 53: nan,
 54: 29,
 55: 73,
 56: nan,
 57: 57,
 58: nan,
 59: nan,
 60: nan,
 61: 30,
 62: 56,
 63: 6,
 64: 55,
 65: 31,
 66: 74,
 67: 54,
 68: 32,
 69: 5,
 70: 33,
 71: 34,
 72: nan,
 73: 53,
 74: 35,
 75: nan,
 76: 36,
 77: 52,
 78: 51,
 79: 37,
 80: 50,
 81: nan,
 82: 38,
 83: 4,
 84: 39,
 85: nan,
 86: 49,
 87: nan,
 88: 40,
 89: 41,
 90: 48,
 91: 47,
 92: 42,
 93: nan,
 94: 43,
 95: 46,
 96: 44,
 97: 45,
 98: 75,
 99: 3,
 100: 2,
 101: nan,
 102: nan,
 103: nan,
 104: nan,
 105: nan,
 106: 1,
 107: nan,
 108: nan,
 109

click MPL MouseEvent: xy=(457,646) xydata=(1326.42194093,592.827004219) button=1 dblclick=False inaxes=Axes(0.160555,0.1;0.70389x0.8)
click MPL MouseEvent: xy=(889,535) xydata=(3604.90295359,1178.27004219) button=3 dblclick=False inaxes=Axes(0.160555,0.1;0.70389x0.8)
click MPL MouseEvent: xy=(510,616) xydata=(1605.95780591,751.054852321) button=1 dblclick=False inaxes=Axes(0.160555,0.1;0.70389x0.8)
click MPL MouseEvent: xy=(482,558) xydata=(2426.0097829,869.406730638) button=1 dblclick=False inaxes=Axes(0.125,0.362372;0.775x0.275257)


In [11]:
fo=find_objects(labeled_array)

In [19]:
labeled_array[fo[1]].shape

(2513, 280)

In [18]:
list(fo[1][1])

TypeError: 'slice' object is not iterable

In [48]:
a=np.nan